In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Download Pretrained Weights

In [5]:
!pip install -q keras-bert

In [0]:
!wget -q https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip

In [0]:
!unzip -o chinese_L-12_H-768_A-12.zip

Archive:  chinese_L-12_H-768_A-12.zip
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


# clone

In [2]:
cd drive/My Drive/ntp

/content/drive/My Drive/ntp


In [3]:
!git clone https://github.com/CyberZHG/keras-bert keras-bert

Cloning into 'keras-bert'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 1047 (delta 87), reused 64 (delta 41), pack-reused 913
Receiving objects: 100% (1047/1047), 14.09 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (653/653), done.


In [6]:
ls

 bert/                     'Neural Style Transfer with Eager Execution.ipynb'
'Car Detection'/           'NLP in tensorflow'/
 chinese_L-12_H-768_A-12/   pycorrector/
 eight/                     pytorch_transformers/
 glue/                      Tom-Chang-Deep-Lyrics-master/
 keras-bert/                udacity_intro_to_tensorflow_for_deep_learning/
 machinetranslation/        最重要的事.docx


# Build Model & Dictionary

In [0]:
import os

pretrained_path = 'chinese_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [0]:
import os

pretrained_path = 'chinese_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [0]:
os.environ['TF_KERAS'] = '1'

In [0]:
import codecs

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
token_dict_inv = {v: k for k, v in token_dict.items()}

In [10]:
from keras_bert import load_trained_model_from_checkpoint

model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True)
model.summary(line_length=120)

W0809 12:01:57.572898 140186760062848 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0809 12:01:58.222465 140186760062848 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               [(None, 512)]              0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             [(None, 512)]              0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 512, 768), (21128, 16226304      Input-Token[0][0]                       
________________________________________________________________________________________________________________________
Embedding-Segment

# Predict Masked

In [40]:
from keras_bert import Tokenizer

tokenizer = Tokenizer(token_dict)
text = '等到潮水退了，就知道誰沒穿褲子'
tokens = tokenizer.tokenize(text)
#tokens[3] = '[MASK]'
print('Tokens:', tokens)

Tokens: ['[CLS]', '等', '到', '潮', '水', '退', '了', '，', '就', '知', '道', '誰', '沒', '穿', '褲', '子', '[SEP]']


In [0]:
import numpy as np

indices = np.array([[token_dict[token] for token in tokens] + [0] * (512 - len(tokens))])
segments = np.array([[0] * len(tokens) + [0] * (512 - len(tokens))])
masks = np.array([[0, 1, 1] + [0] * (512 - 3)])

In [20]:
text_len = len(text)
text_len

15

In [35]:
predicts = model.predict([indices, segments, masks])[0].argmax(axis=-1).tolist()
sentences= list(map(lambda x: token_dict_inv[x], predicts[0][3:4]))
print('Fill with: ', ''.join(sentences))

Fill with:  汗


# My Code

In [89]:
text = '吳光勝打開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。'
text_wrong = 0
for i in range(0,len(tokens)):
  tokens = tokenizer.tokenize(text)
  if tokens[i] in ['，','。','、','[CLS]','[SEP]','：','"','！']:
      continue
  tokens[i] = '[MASK]'
  # print(tokens)
  indices = np.array([[token_dict[token] for token in tokens] + [0] * (512 - len(tokens))])
  segments = np.array([[0] * len(tokens) + [0] * (512 - len(tokens))])
  masks = np.array([[0, 1, 1] + [0] * (512 - 3)])
  
  predicts = model.predict([indices, segments, masks])[0].argmax(axis=-1).tolist()
  sentences= list(map(lambda x: token_dict_inv[x], predicts[0][i:i+1]))
  if ''.join(sentences) != text[i-1] :
    print(''.join(tokens[1:len(tokens)-1]), '\n建議字: ', ''.join(sentences))
    text_wrong += 1
print('預測錯字數目:',text_wrong,'錯誤率',text_wrong/len(tokens))    

[MASK]光勝打開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  李
吳[MASK]勝打開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  得
吳光[MASK]打開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  榮
吳光勝[MASK]開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  推
吳光勝打開[MASK]門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  木
吳光勝打開房門，[MASK]游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  將
吳光勝打開房門，鬱[MASK]標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  元
吳光勝打開房門，鬱游[MASK]在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  舟
吳光勝打開房門，鬱游標在他[MASK]上重重一推

# load my model

In [93]:
ls

 bert/                'Neural Style Transfer with Eager Execution.ipynb'
'Car Detection'/      'NLP in tensorflow'/
 eight/                pycorrector/
 glue/                 pytorch_transformers/
 keras-bert/           Tom-Chang-Deep-Lyrics-master/
 machinetranslation/   udacity_intro_to_tensorflow_for_deep_learning/
 my_chinese_lm/        最重要的事.docx


In [0]:
pretrained_path = 'my_chinese_lm'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [0]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
token_dict_inv = {v: k for k, v in token_dict.items()}

In [98]:
model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True)
model.summary(line_length=120)

Model: "model_1"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               [(None, 512)]              0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             [(None, 512)]              0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 512, 768), (21128, 16226304      Input-Token[0][0]                       
________________________________________________________________________________________________________________________
Embedding-Segme

In [0]:
tokenizer = Tokenizer(token_dict)

In [0]:
indices = np.array([[token_dict[token] for token in tokens] + [0] * (512 - len(tokens))])
segments = np.array([[0] * len(tokens) + [0] * (512 - len(tokens))])
masks = np.array([[0, 1, 1] + [0] * (512 - 3)])

In [101]:
text = '吳光勝打開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。'
text_wrong = 0
for i in range(0,len(tokens)):
  tokens = tokenizer.tokenize(text)
  if tokens[i] in ['，','。','、','[CLS]','[SEP]','：','"','！']:
      continue
  tokens[i] = '[MASK]'
  # print(tokens)
  indices = np.array([[token_dict[token] for token in tokens] + [0] * (512 - len(tokens))])
  segments = np.array([[0] * len(tokens) + [0] * (512 - len(tokens))])
  masks = np.array([[0, 1, 1] + [0] * (512 - 3)])
  
  predicts = model.predict([indices, segments, masks])[0].argmax(axis=-1).tolist()
  sentences= list(map(lambda x: token_dict_inv[x], predicts[0][i:i+1]))
  if ''.join(sentences) != text[i-1] :
    print(''.join(tokens[1:len(tokens)-1]), '\n建議字: ', ''.join(sentences))
    text_wrong += 1
print('預測錯字數目:',text_wrong,'錯誤率',text_wrong/len(tokens))   

[MASK]光勝打開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  李
吳[MASK]勝打開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  百
吳光[MASK]打開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  榮
吳光勝[MASK]開房門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  推
吳光勝打開[MASK]門，鬱游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  木
吳光勝打開房門，[MASK]游標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  大
吳光勝打開房門，鬱[MASK]標在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  錦
吳光勝打開房門，鬱游[MASK]在他背上重重一推，推進門內，隨即關上木門，只聽得喀喇一"你們無量劍講理不講？這可不是把我當作了犯人了嗎？無量劍又不是官府，怎能胡亂關人？"可是外面聲息遽然，任他大長氣，心想："既來之，則安之。 
建議字:  隱
吳光勝打開房門，鬱游標在他[MASK]上重重一推